In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, stddev, when, abs

#jdbc connection 
jdbc_url = (
    "jdbc:sqlserver://fraud-detection-synapse-9613.sql.azuresynapse.net:1433;"
    "database=fraudpool;"
    "encrypt=true;"
    "trustServerCertificate=false;"
    "hostNameInCertificate=*.sql.azuresynapse.net;"
    "loginTimeout=30;"
    "user=username;"
    "password=password-here;"
)

#load data from table
spark.conf.set(
    "fs.azure.account.key.storageaccount9613.dfs.core.windows.net",
    "storage-account-key"
)

anomaly_df = spark.read.format("delta").load(
    "abfss://fraud-detection-container@storageaccount9613.dfs.core.windows.net/anomaly_data"
)

#risk score calculation
expected_states = ["CT", "NY", "NJ", "MA"]
potential_fraud_states = ["TX", "CA", "WA", "OH", "FL"]

risk_score_df = anomaly_df.withColumn(
    "risk_score",
    when((col("merchant_id") == "Taco Bell") & (col("amount") > 100), 0.8)
    .when((col("merchant_id") == "Exxon") & (col("amount") > 150), 0.8)
    .when((col("amount") > 1500), 0.9)
    .when((col("location").isin(potential_fraud_states)) & (col("amount") > 500), 0.9)
    .when((col("location").isin(potential_fraud_states)), 0.5)
    .when((col("location").isin(expected_states)) & (col("amount") > 500), 0.7)
    .otherwise(0.1)
)

#update Synapse table
(risk_score_df.write
    .format("com.databricks.spark.sqldw")
    .option("url", jdbc_url)
    .option("forwardSparkAzureStorageCredentials", "true")
    .option("dbtable", "FraudFlags")
    .option("tempDir", "abfss://fraud-detection-container@storageaccount9613.dfs.core.windows.net/synapse_temp")
    .mode("overwrite")
    .save()
)

#sanity check
fraud_count = risk_score_df.filter(col("anomaly_flag") == 1).count()
print(f"Number of fraudulent transactions: {fraud_count}")
fraud_total = risk_score_df.filter(col("anomaly_flag") == 1).agg({"amount": "sum"}).collect()[0][0]
print(f"Total Amount in Fraudulent Transactions (anomaly_flag == 1): ${fraud_total:,.2f}")


risky_count = risk_score_df.filter(col("risk_score") > 0.1).count()
print(f"Number of risky transactions (risk_score > 0.1): {risky_count}")
risky_total = risk_score_df.filter(col("risk_score") > 0.1).agg({"amount": "sum"}).collect()[0][0]
print(f"Total Amount in Risky Transactions (risk_score > 0.1): ${risky_total:,.2f}")


Number of fraudulent transactions: 134
Number of risky transactions (risk_score > 0.1): 350
💰 Total Amount in Risky Transactions (risk_score > 0.1): $406,532.87
⚠️ Total Amount in Fraudulent Transactions (anomaly_flag == 1): $322,928.42
